<div style ="padding: 15px 20px;
    color: #c8102e;
    background: #01216955;             
    display: flex;
    justify-content: space-between;
    align-items: center;">
    <div>
        <img src="flag_mini.png">
    </div>
    <div>
        <h2 style = "padding : 0 10px">10) Afficher une carte de chaleur qui représente le nombre de médailles gagnées par sport (en vertical) en fonction de l'âge moyen des participants à ce sport (en horizontal)</h2>
    </div>
</div>


In [2]:
import pandas as pd
import os
import ast
import numpy as np
import altair as alt

country = 'GBR'

df_results = pd.read_csv(r"{}/archive/olympic_results.csv".format(os.getcwd()))
df_medals = pd.read_csv(r"{}/archive/olympic_medals.csv".format(os.getcwd()))
df_athletes = pd.read_csv(r"{}/archive/olympic_athletes.csv".format(os.getcwd()))
df_hosts = pd.read_csv(r"{}/archive/olympic_hosts.csv".format(os.getcwd()))

## Récupération de tous les athlètes participants aux JO dont les données sont exploitables ( nom et l'année de naissance sont connus )
## ATTENTION CETTE ÉTAPE PEUT PRENDRE PLUSIEURS MINUTES ! 
**Passer directement à la case suivante pour charger le csv résultant de l'opération ci-dessous.**

In [ ]:
#fusionner df_results & df_hosts pour filter les JO d'étés, puis filtrer
df = df_results.merge(df_hosts, left_on='slug_game', right_on='game_slug')
df.query("game_season == 'Summer' ", inplace=True)

#df_team : récupération des athlètes qui participent en équipe
df_team = df[(df.participant_type == 'GameTeam')]
df_indiv = df[(df.participant_type == 'Athlete')]

#suppression des lignes où les participants de l'épreuve par équipe sont inconnus 
df_team = df_team.drop(df_team[df_team['athletes'].isnull()].index)

#extractions des participants par équipe qui étaient dans des listes et ajout aux autres partipants 
df = df_indiv.copy()
liste_participant=[]
for index,row in df_team.iterrows():
    l = ast.literal_eval(row['athletes'])
    for participant in l :
        df_new_line =  pd.DataFrame([[row['slug_game'],row['participant_type'],row['game_year'],row['athletes'],participant[1]]],columns=['slug_game','participant_type', 'game_year','athletes','athlete_url'])
        liste_participant.append(participant[0])
        df = pd.concat([df, df_new_line]) 

#suppression des lignes où l'athlète n'a pas d'identifiant unique / athlete_url
df.drop(df[df['athlete_url'].isnull()].index, inplace=True)

#éliminations des doublons correspondant aux athlètes présents plusieurs fois la même année 
df.drop_duplicates(subset=['athlete_url','game_year'], keep='first', inplace=True)

#Fusion de notre tableau filtré avec le tableau d'info des athlètes pour obtenir l'année de naissance
df = df.merge(df_athletes, left_on='athlete_url', right_on='athlete_url')

#suppression des lignes où la date de naissance est inconnue
df.drop(df[df['athlete_year_birth'].isnull()].index, inplace=True)

#df.to_csv(r"{}/archive/all.csv".format(os.getcwd()))
print("Après un nettoyage de donnée, nous obtenons une liste de {} présences d'athlètes toutes olympiades confondues, dont la date de naissance est connue.".format(len(df)))


In [3]:
df = pd.read_csv(r"{}/archive/all.csv".format(os.getcwd()))
print("Nous travaillerons une liste de {} présences d'athlètes toutes olympiades confondues, dont la date de naissance est connue.".format(len(df)))


Nous travaillerons une liste de 73978 présences d'athlètes toutes olympiades confondues, dont la date de naissance est connue.


### Calcul de l'âge de tous les athlètes lors de leurs participations aux JO d'étés

In [4]:
df['age']= df.apply(lambda row : ( row['game_year'] - row['athlete_year_birth']), axis=1)
df.head()[['athlete_full_name_x','age']]

,athlete_full_name_x,age
0,Dongqi CHEN,32.0
1,Sagen MADDALENA,27.0
2,Maria MARTYNOVA,23.0
3,Yulia KARIMOVA,26.0
4,Najmeh KHEDMATI,24.0


### Calcul de l'âge moyen de la discipline pour chaque année dans un dataframe à part

In [5]:
# Renommage, regroupement des disciplines
discipline_map={
    "Gymnastics Artistic":"Artistic Gymnastics",
    "Gymnastics Rhythmic":"Rhythmic Gymnastics",
    "Synchronized Swimming":"Artistic Swimming",
    "Equestrian Dressage":"Equestrian",
    "Equestrian Jumping":"Equestrian",
    "Equestrian Eventing":"Equestrian",
    "Trampoline":"Trampoline Gymnastics",
    "Cycling BMX":"Cycling BMX Racing",
    "Short Track Speed Skating": "Short Track"
}
df['discipline_title'] = df['discipline_title'].replace(discipline_map)

#Nouveau dataframe donnant la moyenne d'âge des participants par épreuves et année
df_mean_age = df.groupby(['discipline_title','game_year'])['age'].mean().reset_index(name="mean_age")
df_mean_age.head(100)


,discipline_title,game_year,mean_age
0,Archery,1900,35.666667
1,Archery,1904,47.000000
2,Archery,1908,42.888889
3,Archery,1920,46.333333
4,Archery,1972,31.727273
...,...,...,...
95,Boxing,1956,23.095238
96,Boxing,1960,23.155102
97,Boxing,1964,23.107438
98,Boxing,1968,22.903915


### Pour plus de clarté lors de la visualisation finale, nous arrondissons les âges moyens 

In [6]:
df_mean_age['mean_age'] = round(df_mean_age['mean_age'])
df_mean_age.head()

,discipline_title,game_year,mean_age
0,Archery,1900,36.0
1,Archery,1904,47.0
2,Archery,1908,43.0
3,Archery,1920,46.0
4,Archery,1972,32.0


### Création du dataframe final de toutes les médailles obtenus par la Grande-Bretagne

In [7]:
#fusionner df_medals & df_hosts pour connaître les Jo d'étés
df_final = df_results.merge(df_hosts, left_on='slug_game', right_on='game_slug')

#filtrer : on garde les athlètes de GB médaillés au jeux d'étés
df_final.query("country_3_letter_code == @country and game_season == 'Summer'", inplace=True)
df_final.dropna(subset=['medal_type'], inplace = True)

# Renommage, regroupement des disciplines
df_final['discipline_title'] = df_final['discipline_title'].replace(discipline_map)

print('On récupère une liste de {} médaillés'.format(df_final.shape[0]))


On récupère une liste de 911 médaillés


### Ajout d'une colonne age moyen de la discipline dans le dataframe final de médaillés britanniques


In [9]:
def get_mean_age(discipline,year) :  
    result = df_mean_age.loc[(df_mean_age['discipline_title'] == discipline) & (df_mean_age['game_year'] == year)]['mean_age']
    # try & catch pour les cas où aucune moyenne correspondante est trouvée
    try :
        return(result.iloc[0])
    except IndexError as e:
        print("Aucune moyenne de trouver pour : ",discipline, year)
        return None

df_final['discipline_mean_age'] = df_final.apply(lambda row : get_mean_age(row['discipline_title'], row['game_year']), axis = 1)

Aucune moyenne de trouver pour :  Hockey 2020
Aucune moyenne de trouver pour :  Rugby 2016
Aucune moyenne de trouver pour :  Hockey 2016
Aucune moyenne de trouver pour :  Hockey 2012
Aucune moyenne de trouver pour :  Hockey 1992
Aucune moyenne de trouver pour :  Hockey 1988
Aucune moyenne de trouver pour :  Hockey 1984
Aucune moyenne de trouver pour :  Hockey 1952
Aucune moyenne de trouver pour :  Hockey 1948
Aucune moyenne de trouver pour :  Sailing 1936
Aucune moyenne de trouver pour :  Polo 1936
Aucune moyenne de trouver pour :  Polo 1924
Aucune moyenne de trouver pour :  Hockey 1920
Aucune moyenne de trouver pour :  Polo 1920
Aucune moyenne de trouver pour :  Sailing 1920
Aucune moyenne de trouver pour :  Tug of War 1920
Aucune moyenne de trouver pour :  Water Polo 1920
Aucune moyenne de trouver pour :  Football 1912
Aucune moyenne de trouver pour :  Tug of War 1912
Aucune moyenne de trouver pour :  Water Polo 1912
Aucune moyenne de trouver pour :  Rugby 1908
Aucune moyenne de trou

In [14]:
df_final.head(15)[['athlete_full_name','discipline_title','game_year','discipline_mean_age']]

,athlete_full_name,discipline_title,game_year,discipline_mean_age
4019,Matthew John COWARD HOLLEY,Shooting,2020,30.0
4311,NaN,Diving,2020,24.0
4321,Jack LAUGHER,Diving,2020,24.0
4396,Thomas DALEY,Diving,2020,24.0
4708,Liam HEATH,Canoe Sprint,2020,26.0
5026,Pat MCCORMACK,Boxing,2020,26.0
5072,Frazer CLARKE,Boxing,2020,26.0
5089,Benjamin WHITTAKER,Boxing,2020,26.0
5141,Lauren PRICE,Boxing,2020,26.0
5226,Galal YAFAI,Boxing,2020,26.0


## Affichage de la carte de chaleur

In [19]:
source = df_final

# Configure common options
base = alt.Chart(source).transform_aggregate(
    participants='count()',
    groupby=['discipline_mean_age', 'discipline_title']
).encode(
    alt.X('discipline_mean_age:O', scale=alt.Scale(paddingInner=0), title="Age moyen des participants"),
    alt.Y('discipline_title:O', scale=alt.Scale(paddingInner=0), title="Disciplines"),
).properties(title = "Médailles gagnées en fonction de l'âge moyen des participants")

# Configure heatmap
heatmap = base.mark_rect().encode(
    color=alt.Color('participants:Q',
        scale=alt.Scale(scheme='viridis'),
        legend=alt.Legend(direction='horizontal')
    )
)

# Configure text
text = base.mark_text(baseline='middle').encode(
    text='participants:Q',
    color=alt.condition(
        alt.datum.participants > 100,
        alt.value('black'),
        alt.value('white')
    )
)

# Changer le nombre de ligne à intégrer ( par défaut = 5000 )
alt.data_transformers.enable('default', max_rows=5500)

DataTransformerRegistry.enable('default')

# Résultats : 

In [20]:
heatmap + text

alt.LayerChart(...)